In [18]:
mapa_metabolico ={
    
    #Glicólise

    "glicose": {
        "glicose-6-fosfato": {"label": "hexocinase"}
    },
    
    "glicose-6-fosfato": {
        "frutose-6-fosfato": {"label": "fosfo-hexose-isomerase"}
    },
    
    "frutose-6-fosfato": { 
        "glicose-6-fosfato": {"label": "fosfo-hexose-isomerase"},
        "frutose-1,6-bisfosfato": {"label": "fosfofrutocinase 1"}
    },
    
    "frutose-1,6-bisfosfato": {
        "gliceraldeído-3-fosfato + di-hidroxiacetona-fosfato": {"label": "aldolase"}
    },

    "gliceraldeído-3-fosfato + di-hidroxiacetona-fosfato": {
        "frutose-1,6-bisfosfato": {"label": "aldolase"},
        "(2) gliceraldeído-3-fosfato": {"label": "triose-fosfato-isomerase"}
    },

    "(2) gliceraldeído-3-fosfato": {
        "gliceraldeído-3-fosfato + di-hidroxiacetona-fosfato": {"label": "triose-fosfato-isomerase"},
        "(2) 1,3-bisfosfoglicerato": {"label": "gliceraldeido-3-fosfato-desidrogenase"}
    },

    "(2) 1,3-bisfosfoglicerato": {
        "(2) gliceraldeído-3-fosfato": {"label": "gliceraldeido-3-fosfato-desidrogenase"},
        "(2) 3-fosfoglicerato": {"label": "fosfoglicerato-cinase"}
    },

    "(2) 3-fosfoglicerato": {
        "(2) 1,3-bisfosfoglicerato": {"label": "osfoglicerato-cinase"},
        "(2) 2-fosfoglicerato": {"label": "fosfoglicerato-mutase"}
    },

    "(2) 2-fosfoglicerato":{
        "(2) 3-fosfoglicerato": {"label": "fosfoglicerato-mutase"},
        "(2) fosfoenolpiruvato": {"label": "enolase"}
    },

    "(2) fosfoenolpiruvato": {
        "(2) 2-fosfoglicerato": {"label": "enolase"},
        "(2) piruvato": {"label": "piruvato-cinase"}
    },
    #Fermentação 

    "(2) piruvato": {"lactato": {"label": "(lática) lactato desidrogenase"},
        "NAD+": {"label": "lactato desidrogenase"},
        "acetaldeído": {"label": "(alcóolica) piruvato descarboxilase"},
        "CO2": {"label": "(alcóolica) piruvato descarvoxilase"},
        "acetil-CoA" : {"label": "piruvato-desidrogenase"}
    },
    "acetaldeído": {
        "etanol": {"label": "álcool desidrogenase"},
        "NAD+": {"label": "álcool desidrogenase"}
    },

    #Ciclo de Krebs
    "acetil-CoA": {
        "citrato": {"label": "citrato-sintetase "},
        "malonil-CoA": {"label": "acetil-CoA carboxilase (ACC)"},
    },
    "citrato": {
        "cis-aconitato": {"label": "aconitase"},
        
    },
    "cis-aconitato":{
        "isocitrato": {"label": "aconitase"},
        
    },
    "isocitrato": {
        "oxalosuccinato": {"label":"isocitrato-desidrogenase"},
        
    },
    "oxalosuccinato":{
        "a-cetoglutarato": {"label": "isocitrato-desidrogenase"},
     },
    "a-cetoglutarato": {
        "succinil-CoA": {"label": "a-cetoglutarato-desidrogenase"},
    
    },
    "succinil-CoA": {
        "succinato": {"label": "succinil-CoA-Sintetase"},
    
    },
    "succinato": {
        "fumarato": {"label": "succinato-desidrogenase"},
    
    },
    "fumarato": {
        "L-malato": {"label": "fumarase"},
    
    }, 
    "L-malato": {
        "oxaloacetato": {"label": "malato-desidrogenase"},
    
    },
    "oxaloacetato": {
        "citrato": {"label": "citratase"}
    
    },
    #Síntese de ácidos graxos:
    "malonil-CoA": {
        "palmitato": {"label": "ácido graxo sintase"}
    },
    "palmitato": {
        "palmitoil-CoA": {"label": "ácido graxo sintase"},
        "acetil-CoA": {"label": "ácido graxo sintase"}
    }
}

In [19]:
def busca_em_profundidade_direcionada(vertice_inicial, vertice_final, grafo, visitados=None):
    if visitados is None:
        visitados = []

    visitados.append(vertice_inicial)

    if vertice_inicial == vertice_final:
        return visitados

    for vizinho in grafo.get(vertice_inicial, {}):
        if vizinho not in visitados:
            resultado = busca_em_profundidade_direcionada( vizinho, vertice_final, grafo, visitados.copy())
            if resultado and resultado[-1] == vertice_final:
                return resultado

    return None

In [20]:
print(busca_em_profundidade_direcionada("frutose-6-fosfato", "gliceraldeído-3-fosfato + di-hidroxiacetona-fosfato", mapa_metabolico))

['frutose-6-fosfato', 'frutose-1,6-bisfosfato', 'gliceraldeído-3-fosfato + di-hidroxiacetona-fosfato']


# Transformação da rota em grafo:
Cria um novo grafo ponderado pelas enzimas da rota metabólica a partir da saída da função busca_em_profundidade_direcionada. A cauda da aresta é um item da lista de saída, a cabeça é o item seguinte e o peso da aresta é a enzima correspondente à cauda.

In [84]:
def rota_em_grafo(vertice_inicial, vertice_final, grafo):
    rota = busca_em_profundidade_direcionada(vertice_inicial, vertice_final, grafo)
    novo_grafo = {}
    try:
        for i in range(len(rota)):
            for enzima in grafo[rota[i]]:
                if i+1 < len(rota):
                    novo_grafo[rota[i]] = {rota[i+1]: {"label": grafo[rota[i]][enzima]['label']}}
                elif i+1 == len(rota):
                    novo_grafo[rota[i]] = {}
    except:
        return "Não é possível realizar a reação."

    return novo_grafo

In [85]:
rota_em_grafo("glicose-6-fosfato", "frutose-1,6-bisfosfato", mapa_metabolico)

{'glicose-6-fosfato': {'frutose-6-fosfato': {'label': 'fosfo-hexose-isomerase'}},
 'frutose-6-fosfato': {'frutose-1,6-bisfosfato': {'label': 'fosfofrutocinase 1'}},
 'frutose-1,6-bisfosfato': {}}

In [86]:
def metabolitos_e_enzimas(vertice_inicial, vertice_final, grafo):
    rota_com_enzimas = rota_em_grafo(vertice_inicial, vertice_final, grafo)
    if type(rota_com_enzimas) != dict:
        print(f"Não é possível chegar ao {vertice_final} partindo do {vertice_inicial}")
    else:
        print(f"Para obter o(a) {vertice_final} a partir do(a) {vertice_inicial}, é preciso passar pela rota:") 
        lista_da_rota = list(rota_com_enzimas.keys())
        lista_grafo = list(grafo.keys())
        if lista_grafo.index(vertice_inicial) < lista_grafo.index(vertice_final):
            for i in range(len(lista_da_rota) - 1):
                print(f"Metabolito: {lista_da_rota[i]} -> [Enzima: {rota_com_enzimas[lista_da_rota[i]][lista_da_rota[i+1]]["label"]}] ->")
        else: 
             for i in range(len(lista_da_rota) - 1):
                print(f"Metabolito: {lista_da_rota[i]} -> [Enzima: {rota_com_enzimas[lista_da_rota[i+1]][lista_da_rota[i+2]]["label"]} ->")

        print(f"Produto: {lista_da_rota[-1]}")

        

In [87]:
metabolitos_e_enzimas("glicose-6-fosfato", "frutose-1,6-bisfosfato", mapa_metabolico)

Para obter o(a) frutose-1,6-bisfosfato a partir do(a) glicose-6-fosfato, é preciso passar pela rota:
Metabolito: glicose-6-fosfato -> [Enzima: fosfo-hexose-isomerase] ->
Metabolito: frutose-6-fosfato -> [Enzima: fosfofrutocinase 1] ->
Produto: frutose-1,6-bisfosfato
